### import lib and functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import os
import glob
import nibabel
import imageio
from scipy import misc, ndimage
from skimage import io, exposure, img_as_uint
from sklearn.model_selection import KFold
from srmodel import *

Using TensorFlow backend.


## Directories

In [2]:
npydir = '/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/512_patch'
if not os.path.exists(npydir):
    os.makedirs(npydir)

patchdir = '/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch'
if not os.path.exists(patchdir):
    os.makedirs(patchdir)
    
testdir = '/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/nii'
if not os.path.exists(testdir):
    os.makedirs(testdir)
    
modeldir = '/home/raynard/Documents/models/3DUnetCNN-master/model/'
if not os.path.exists(modeldir):
    os.makedirs(modeldir)

# Slice nii.gz files in Z direction

In [ ]:
orig_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/orig/*.nii.gz")
count = 0
for filepath in orig_files:
    file = nibabel.load(filepath).get_data().transpose(1, 0, 2)
    z = file.shape[2]
    n_patch = z//33
    j = 0
    for i in range(n_patch):
        patch = file[:,:,j:j+32]
        patch = patch.astype(np.int16)
        print(patch.shape)
        filename_ = filepath.split('/')[-1].split('.')[0]
        filename = filename_ + str(i) + '_OP.npy'
        file1 = os.path.join(npydir, filename)
        np.save(file1, patch)
        count += 1
        j += 33
    print(count)

# Slice nii.gz files in X-Y direction

In [ ]:
op_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/512_patch/*_OP.npy")
for filepath in op_files:
    patch = np.load(filepath)
    x = np.split(patch, 4, axis = 0)
    a = np.split(x[0], 4, axis = 1)
    b = np.split(x[1], 4, axis = 1)
    c = np.split(x[2], 4, axis = 1)
    d = np.split(x[3], 4, axis = 1)
    
    for i in range(4):
        filename_ = filepath.split('/')[-1].split('OP.')[0]
        filename = filename_ + 'a{}_patch.npy'.format(i)
        path = os.path.join(patchdir, filename)
        np.save(path, a[i])
        
    for i in range(4):
        filename_ = filepath.split('/')[-1].split('OP.')[0]
        filename = filename_ + 'b{}_patch.npy'.format(i)
        path = os.path.join(patchdir, filename)
        np.save(path, b[i])
        
    for i in range(4):
        filename_ = filepath.split('/')[-1].split('OP.')[0]
        filename = filename_ + 'c{}_patch.npy'.format(i)
        path = os.path.join(patchdir, filename)
        np.save(path, c[i])
        
    for i in range(4):
        filename_ = filepath.split('/')[-1].split('OP.')[0]
        filename = filename_ + 'd{}_patch.npy'.format(i)
        path = os.path.join(patchdir, filename)
        np.save(path, d[i])

## Check CT patches in nii.gz format

In [ ]:
patch_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*patch.npy")
for filepath in patch_files:
    patch = np.load(filepath)
    
    filename_ = filepath.split('/')[-1].split('.')[0]
    og_file = filename_.split('_CT')[0]
    filename = filename_ + '.nii.gz'
    file1 = os.path.join(testdir, filename)
    
    orig = nibabel.load('/home/raynard/Documents/models/3DUnetCNN-master/data/orig/{}_CT.nii.gz'.format(og_file))
    header = orig.header.copy()
    patch = np.transpose(patch, (1,0,2))
    patch = nibabel.nifti1.Nifti1Image(patch, None, header = header)
    patch.to_filename(file1)

# Prepare training data

In [ ]:
patch_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*patch.npy")
for filepath in patch_files:
    patch = np.load(filepath)
    train_patch = patch[:,:,1::2]
    filename_ = filepath.split('/')[-1].split('patch.')[0]
    filename = filename_ + 'train.npy'
    path = os.path.join(patchdir, filename)
    np.save(path, train_patch)

## Check training data in nii.gz format

In [ ]:
train_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*train.npy")
for filepath in train_files:
    patch = np.load(filepath)
    
    filename_ = filepath.split('/')[-1].split('.')[0]
    og_file = filename_.split('_CT')[0]
    filename = filename_ + '.nii.gz'
    file1 = os.path.join(testdir, filename)
    
    orig = nibabel.load('/home/raynard/Documents/models/3DUnetCNN-master/data/orig/{}_CT.nii.gz'.format(og_file))
    header = orig.header.copy()
    patch = np.transpose(patch, (1,0,2))
    patch = nibabel.nifti1.Nifti1Image(patch, None, header = header)
    patch.to_filename(file1)

# Functions

In [ ]:
def get_input(path):
    lr_img = np.load(path)
    lr_img = lr_img.astype(np.float64)
    lr_img = np.reshape(lr_img, lr_img.shape + (1,))
    return(lr_img)

def get_output(path):
    img_id = path.replace('train','patch')
    hr_img = np.load(img_id)
    hr_img = hr_img.astype(np.float64)
    hr_img = np.reshape(hr_img, hr_img.shape + (1,))
    return(hr_img)

def data_generator(files, batch_size = 4):
    while True:
        batch_paths = np.random.choice(a = files, 
                                         size = batch_size)
        batch_input = []
        batch_output = [] 
        for input_path in batch_paths:
            input = get_input(input_path )
            output = get_output(input_path)
            batch_input += [ input ]
            batch_output += [ output ]
        
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        yield( batch_x, batch_y )

# 4 Fold CV

In [ ]:
model = unet()
model.summary()
model = None

In [ ]:
data = glob.glob('/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*train.npy')
data.sort()
data = np.asarray(data)

kf = KFold(n_splits=4, shuffle=True, random_state=42) 
kf.get_n_splits(data) 
i = 0

for train_index, test_index in kf.split(data):
    if i == 0:
        print("Training Fold:", i)
        x_train, x_test = data[train_index], data[test_index]
        train_gen = data_generator(x_train, batch_size = 2)
        val_gen = data_generator(x_test, batch_size = 2)
        model = None
        model = unet()
        model = load_model(modeldir + 'unet_CV_{}.hdf5'.format(i))
        model_checkpoint = ModelCheckpoint(modeldir + 'unet_CV_{}.hdf5'.format(i), monitor='val_loss',verbose=1, save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1,
                                      patience=5, min_lr=1e-6)
        model.fit_generator(train_gen, steps_per_epoch=4000, epochs=10, verbose =1, callbacks=[model_checkpoint, reduce_lr], validation_data=val_gen, validation_steps=1000, shuffle=True)
        i += 1
    else:
        i += 1

In [ ]:
data = glob.glob('/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*train.npy')
data.sort()
data = np.asarray(data)

kf = KFold(n_splits=4, shuffle=True, random_state=42) 
kf.get_n_splits(data) 
i = 0

for train_index, test_index in kf.split(data):
    if i == 1:
        print("Training Fold:", i)
        x_train, x_test = data[train_index], data[test_index]
        train_gen = data_generator(x_train, batch_size = 2)
        val_gen = data_generator(x_test, batch_size = 2)
        model = None
        model = unet()
        model = load_model(modeldir + 'unet_CV_{}.hdf5'.format(i))
        model_checkpoint = ModelCheckpoint(modeldir + 'unet_CV_{}.hdf5'.format(i), monitor='val_loss',verbose=1, save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1,
                                      patience=5, min_lr=1e-6)
        model.fit_generator(train_gen, steps_per_epoch=4000, epochs=10, verbose =1, callbacks=[model_checkpoint, reduce_lr], validation_data=val_gen, validation_steps=1000, shuffle=True)
        i += 1
    else:
        i += 1

In [ ]:
data = glob.glob('/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*train.npy')
data.sort()
data = np.asarray(data)

kf = KFold(n_splits=4, shuffle=True, random_state=42) 
kf.get_n_splits(data) 
i = 0

for train_index, test_index in kf.split(data):
    if i == 2:
        print("Training Fold:", i)
        x_train, x_test = data[train_index], data[test_index]
        train_gen = data_generator(x_train, batch_size = 2)
        val_gen = data_generator(x_test, batch_size = 2)
        model = None
        model = unet()
        model = load_model(modeldir + 'unet_CV_{}.hdf5'.format(i))
        model_checkpoint = ModelCheckpoint(modeldir + 'unet_CV_{}.hdf5'.format(i), monitor='val_loss',verbose=1, save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1,
                                      patience=5, min_lr=1e-6)
        model.fit_generator(train_gen, steps_per_epoch=4000, epochs=10, verbose =1, callbacks=[model_checkpoint, reduce_lr], validation_data=val_gen, validation_steps=1000, shuffle=True)
        i += 1
    else:
        i += 1

In [ ]:
data = glob.glob('/home/raynard/Documents/models/3DUnetCNN-master/data/CT_Patches/128_patch/*train.npy')
data.sort()
data = np.asarray(data)

kf = KFold(n_splits=4, shuffle=True, random_state=42) 
kf.get_n_splits(data) 
i = 0

for train_index, test_index in kf.split(data):
    if i == 3:
        print("Training Fold:", i)
        x_train, x_test = data[train_index], data[test_index]
        train_gen = data_generator(x_train, batch_size = 2)
        val_gen = data_generator(x_test, batch_size = 2)
        model = None
        model = unet()
        model = load_model(modeldir + 'unet_CV_{}.hdf5'.format(i))
        model_checkpoint = ModelCheckpoint(modeldir + 'unet_CV_{}.hdf5'.format(i), monitor='val_loss',verbose=1, save_best_only=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, verbose=1,
                                      patience=5, min_lr=1e-6)
        model.fit_generator(train_gen, steps_per_epoch=4000, epochs=10, verbose =1, callbacks=[model_checkpoint, reduce_lr], validation_data=val_gen, validation_steps=1000, shuffle=True)
        i += 1
    else:
        i += 1

# Slice CT's into individual folders

In [3]:
test_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/test/*.nii.gz")
count = 0
for filepath in test_files:
    filename = filepath.split('/')[-1].split('.')[0]
    ct512dir = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/512'.format(filename)
    if not os.path.exists(ct512dir):
        os.makedirs(ct512dir)
    
    ct128dir = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/128'.format(filename)
    if not os.path.exists(ct128dir):
        os.makedirs(ct128dir)
    
    file = nibabel.load(filepath).get_data().transpose(1, 0, 2)
    z = file.shape[2]
    n_patch = z//33
    j = 0
    for i in range(n_patch):
        patch = file[:,:,j:j+32]
        patch = patch.astype(np.int16)
        filename = str(i) + '.npy'
        file1 = os.path.join(ct512dir, filename)
        np.save(file1, patch)
        x = np.split(patch, 4, axis = 0)
        a = np.split(x[0], 4, axis = 1)
        b = np.split(x[1], 4, axis = 1)
        c = np.split(x[2], 4, axis = 1)
        d = np.split(x[3], 4, axis = 1)

        for i in range(4):
            filename_ = filename.split('.')[0]
            filename_ = filename_ + '_a{}_ct.npy'.format(i)
            path = os.path.join(ct128dir, filename_)
            np.save(path, a[i])

        for i in range(4):
            filename_ = filename.split('.')[0]
            filename_ = filename_ + '_b{}_ct.npy'.format(i)
            path = os.path.join(ct128dir, filename_)
            np.save(path, b[i])

        for i in range(4):
            filename_ = filename.split('.')[0]
            filename_ = filename_ + '_c{}_ct.npy'.format(i)
            path = os.path.join(ct128dir, filename_)
            np.save(path, c[i])

        for i in range(4):
            filename_ = filename.split('.')[0]
            filename_ = filename_ + '_d{}_ct.npy'.format(i)
            path = os.path.join(ct128dir, filename_)
            np.save(path, d[i])
        count += 1
        j += 33

In [4]:
test_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/test/*.nii.gz")

model = None
model = unet()
model = load_model(modeldir + 'unet_CV_0.hdf5')

for filepath in test_files:
    filename = filepath.split('/')[-1].split('.')[0]
    og_file = filename
    ct128dir = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/128'.format(filename)
    files = glob.glob(ct128dir + "/*_ct.npy")
    
    for path in files:
        patch = np.load(path)
        
        sr_input = patch[:,:,1::2]
        sr_input = sr_input.astype(np.float64)
        sr_input = np.reshape(sr_input, (1,) + sr_input.shape + (1,)) 
        sr_patch = model.predict(sr_input)
        sr_patch = np.squeeze(sr_patch)
        sr_patch = sr_patch.astype(np.int16)
        
        sr_name = path.split('/')[-1].split('_ct.')[0]
        sr_name = sr_name + '_sr.npy'
        file2 = os.path.join(ct128dir, sr_name)

        np.save(file2, sr_patch)

In [ ]:
from skimage.measure import compare_ssim, compare_psnr

test_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/test/*.nii.gz")

for filepath in test_files:
    psnr = 0
    ssim = 0
    n = 0
    filename = filepath.split('/')[-1].split('.')[0]
    ct128dir = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/128'.format(filename)
    files = glob.glob(ct128dir + "/*_sr.npy")
    
    for path in files:
        model_patch = np.load(path)
        path_ = path.replace('_sr.npy','_ct.npy')
        patch = np.load(path_)
        patch_ssim = compare_ssim(patch, model_patch)
        patch_psnr = compare_psnr(patch, model_patch)
        ssim += patch_ssim
        psnr += patch_psnr
        n += 1
    
    avg_psnr = psnr / n
    avg_ssim = ssim / n
    print("{} Average PSNR:".format(filename), avg_psnr, "Average SSIM:", avg_ssim)
        
        

# Stitching

In [5]:
test_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/test/*.nii.gz")

for filepath in test_files:
    filename = filepath.split('/')[-1].split('.')[0]
    ct128dir = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/128'.format(filename)
    orig = nibabel.load('/home/raynard/Documents/models/3DUnetCNN-master/data/test/{}.nii.gz'.format(filename))
    outfile1 = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/{}_sr.nii.gz'.format(filename, filename)
    outfile2 = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/{}.nii.gz'.format(filename, filename)
    header = orig.header.copy()
    patches = glob.glob(ct128dir + "/*_sr.npy")
    patches.sort()
    layers = len(patches)/16
    layers = int(layers)
    
    layer = []
    ct_layer = []
    for i in range(layers):
        
        alist = glob.glob(ct128dir + "/{}_a*_sr.npy".format(i))
        alist.sort()
        a = []
        a_ct = []
        for path in alist:
            patch = np.load(path)
            a.append(patch)
            path_ = path.replace('_sr.npy','_ct.npy')
            patch_ = np.load(path_)
            a_ct.append(patch_)
        a_all = np.concatenate((a), axis = 1)
        a_ct_all = np.concatenate((a_ct), axis = 1)
        
        blist = glob.glob(ct128dir + "/{}_b*_sr.npy".format(i))
        blist.sort()
        b = []
        b_ct = []
        for path in blist:
            patch = np.load(path)
            b.append(patch)
            path_ = path.replace('_sr.npy','_ct.npy')
            patch_ = np.load(path_)
            b_ct.append(patch_)
        b_all = np.concatenate((b), axis = 1)
        b_ct_all = np.concatenate((b_ct), axis = 1)
        
        clist = glob.glob(ct128dir + "/{}_c*_sr.npy".format(i))
        clist.sort()
        c = []
        c_ct = []
        for path in clist:
            patch = np.load(path)
            c.append(patch)
            path_ = path.replace('_sr.npy','_ct.npy')
            patch_ = np.load(path_)
            c_ct.append(patch_)
        c_all = np.concatenate((c), axis = 1)
        c_ct_all = np.concatenate((c_ct), axis = 1)
        
        
        dlist = glob.glob(ct128dir + "/{}_d*_sr.npy".format(i))
        dlist.sort()
        d = []
        d_ct = []
        for path in dlist:
            patch = np.load(path)
            d.append(patch)
            path_ = path.replace('_sr.npy','_ct.npy')
            patch_ = np.load(path_)
            d_ct.append(patch_)
        d_all = np.concatenate((d), axis = 1)
        d_ct_all = np.concatenate((d_ct), axis = 1)
        
        combinedlayer = np.concatenate((a_all, b_all, c_all, d_all), axis = 0)
        combinedlayer_ct = np.concatenate((a_ct_all, b_ct_all, c_ct_all, d_ct_all), axis = 0)
        
        layer.append(combinedlayer)
        ct_layer.append(combinedlayer_ct)
        
    modelct = np.dstack(layer)
    ct = np.dstack(ct_layer)
    modelct = np.transpose(modelct, (1,0,2))
    ct = np.transpose(ct, (1,0,2))
    modelct = nibabel.nifti1.Nifti1Image(modelct, None, header = header)
    modelct.to_filename(outfile1)
    ct = nibabel.nifti1.Nifti1Image(ct, None, header = header)
    ct.to_filename(outfile2)


        

In [6]:
from skimage.measure import compare_ssim, compare_mse

test_files = glob.glob("/home/raynard/Documents/models/3DUnetCNN-master/data/test/*.nii.gz")

for filepath in test_files:
    filename = filepath.split('/')[-1].split('.')[0]
    ctpath = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/{}.nii.gz'.format(filename, filename)
    outfile = '/home/raynard/Documents/models/3DUnetCNN-master/data/eval/{}/{}_error.nii.gz'.format(filename, filename)
    orig = nibabel.load(ctpath).get_data().transpose(1, 0, 2)
    modelctpath = ctpath.replace('CT.nii.gz','CT_sr.nii.gz')
    model = nibabel.load(modelctpath).get_data().transpose(1, 0, 2)
    copy = nibabel.load(ctpath)
    header = copy.header.copy()
    
    orig_odd = orig[:,:,1::2]
    orig_even = orig[:,:,::2]
    
    model_odd = model[:,:,1::2]
    model_even = model[:,:,::2]
    
    odd_mse = compare_mse(orig_odd, model_odd)
    even_mse = compare_mse(orig_even, model_even)
    
    odd_ssim = compare_ssim(orig_odd, model_odd)
    even_ssim = compare_ssim(orig_even, model_even)
    
    error = orig - model
    error = np.transpose(error, (1,0,2))
    error = nibabel.nifti1.Nifti1Image(error, None, header = header)
    error.to_filename(outfile)

    print("{} \nOdd MSE: {:07.3f}\tEven MSE: {:07.3f} \nOdd SSIM: {:05.4f}\tEven SSIM: {:05.4f} \n".format(filename, odd_mse, even_mse, odd_ssim, even_ssim))

Feb08_P1603_CT 
Odd MSE: 798.647	Even MSE: 1686.065 
Odd SSIM: 0.9998	Even SSIM: 0.9995 

Feb08_P1733_CT 
Odd MSE: 587.188	Even MSE: 1485.865 
Odd SSIM: 0.9998	Even SSIM: 0.9996 

Feb08_P1633_CT 
Odd MSE: 601.227	Even MSE: 1197.430 
Odd SSIM: 0.9998	Even SSIM: 0.9996 

Feb08_P1773_CT 
Odd MSE: 534.009	Even MSE: 1450.813 
Odd SSIM: 0.9998	Even SSIM: 0.9996 

Feb08_P1873_CT 
Odd MSE: 861.232	Even MSE: 1775.925 
Odd SSIM: 0.9997	Even SSIM: 0.9995 

Feb08_P1643_CT 
Odd MSE: 836.674	Even MSE: 1694.996 
Odd SSIM: 0.9997	Even SSIM: 0.9995 

Feb08_P1403_CT 
Odd MSE: 648.018	Even MSE: 1718.812 
Odd SSIM: 0.9998	Even SSIM: 0.9995 

Feb08_P1673_CT 
Odd MSE: 698.159	Even MSE: 1409.699 
Odd SSIM: 0.9998	Even SSIM: 0.9996 

Feb08_P1783_CT 
Odd MSE: 840.287	Even MSE: 1848.491 
Odd SSIM: 0.9997	Even SSIM: 0.9995 

Feb08_P1763_CT 
Odd MSE: 744.308	Even MSE: 1593.821 
Odd SSIM: 0.9998	Even SSIM: 0.9995 

Feb08_P1813_CT 
Odd MSE: 566.039	Even MSE: 1257.165 
Odd SSIM: 0.9998	Even SSIM: 0.9996 

Feb08_P172